In [0]:
import dataiku
import pandas as pd
from dataiku import pandasutils as pdu

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
OUTPUT_DATASET_NAME = "agent_hub_instances"

# The specific plugin ID for Agent Hub is typically 'dataiku-agent-hub' 
# or 'agent-hub'. We will search loosely for 'agent' and 'hub' in the type 
# to ensure we catch them even if the internal ID varies slightly.
def is_agent_hub(webapp):
    webapp_type = webapp.get('type', '').lower()
    # Agent Hub is a plugin webapp. Its type usually follows the pattern 'plugin_id.webapp_id'
    # We check for the presence of specific keywords to identify it.
    if webapp_type == 'webapp_agent-hub_agent-hub':
        return True
    
    # Also check the name as a fallback if the user named it explicitly "Agent Hub" 
    # and the type is generic (less likely, but robust).
    if 'agent hub' in webapp.get('name', '').lower():
        return True
        
    return False

# -----------------------------------------------------------------------------
# Main Execution
# -----------------------------------------------------------------------------
client = dataiku.api_client()

# 1. Retrieve list of all projects (User is Admin per assumptions)
project_keys = client.list_project_keys()
print(f"Scanning {len(project_keys)} projects for Agent Hub instances...")

agent_hubs = []

for project_key in project_keys:
    try:
        project = client.get_project(project_key)
        
        # List all webapps in the project
        webapps = project.list_webapps()
        
        for webapp in webapps:
            if is_agent_hub(webapp):
                # print(f"Found Agent Hub: {webapp['name']} in {project_key}")
                
                # Collect relevant metadata
                agent_hubs.append({
                    "project_key": project_key,
                    "webapp_name": webapp.get("name"),
                    "webapp_id": webapp.get("id"),
                    # "type": webapp.get("type"),
                    "created_by": webapp.get("createdBy", {}).get("login"),
                    # "creation_date": webapp.get("creationDate"),
                    "url": f"/projects/{project_key}/webapps/{webapp.get('id')}/view"
                })
                
    except Exception as e:
        print(f"Skipping project {project_key} due to error: {e}")

# 2. Convert to DataFrame
df = pd.DataFrame(agent_hubs)

df